<a href="https://colab.research.google.com/github/SonnyK33/test/blob/master/Corporate_Credit_Class_11_24_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
from sqlite3 import Error

# conn = sqlite3.connect("test.db")
# c = conn.cursor()

# c.execute("Create TABLE stocks2 (date text)")

OilPrices = [40, 50, 60, 70]


class CorporateCredit():
  def __init__(self, name, cusip, ticker, rating, sector, country):
    self.name = name
    self.cusip = cusip
    self.ticker = ticker
    self.rating = rating
    self.sector = sector
    self.country = country

  def Print_Description(self):
    print(f"Name: ",self.name)
    print(f"Ticker: ",self.ticker)
    print(f"Rating: ",self.rating)
    print(f"Sector: ",self.sector)
    print(f"Country: ",self.country)

  #def get historical levels

  
  #def settargets
    # sets target prices, yields, spreads
  
  def setCurrentFCF(self, rev, rev_g, var_costs, f_costs, cash, capex_rev, WC_rev, taxes):
      self.CurrentFCF = {'rev': rev, 'rev_g': rev_g, 'var_costs': var_costs, 'f_costs': f_costs, 'cash': cash, 'capex/rev': capex_rev, 'WC/rev': WC_rev, 'tax':taxes}

    

    def makeProj (self, qtrs):
      revArray = np.array([self.CurrentFCF['rev']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
      vcostsArray = np.array([self.CurrentFCF['var_costs']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
      fcostsArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['f_costs'],dtype=int)
      ebitdaArray = revArray+vcostsArray+fcostsArray
      capexArray = revArray * self.CurrentFCF['capex/rev']
      wcArray = revArray * self.CurrentFCF['WC/rev']
      taxArray = ebitdaArray * self.CurrentFCF['tax']  #to fix --> need interest exp
      fcfArray = ebitdaArray + capexArray + wcArray + taxArray # need interest expense

      begCashArray = np.zeros(qtrs+1,dtype=int)
      begCashArray[0] = self.CurrentFCF['cash']
      endCashArray = np.zeros(qtrs+1,dtype=int)
      endCashArray[0] = fcfArray[0]+begCashArray[0]
      

      for i in range(1,qtrs+1):
        begCashArray[i] = endCashArray[i-1]        
        endCashArray[i] = begCashArray[i] + fcfArray[i]
        
      
      FCF_statement=[revArray,vcostsArray,fcostsArray,ebitdaArray,capexArray,wcArray,taxArray,fcfArray,begCashArray,endCashArray]
      FCF_df=pd.DataFrame(FCF_statement,columns = self.makeDateColumn(qtrs), index = FCF, dtype=int)

      # print(ebitdaArray)
      # print(begCashArray)
      # print(endCashArray)
      

      return FCF_df


      
    def makeDateColumn(self,qtrs):
      yearColumns = ['Q'+str(currentQuarter)+'/'+str(math.floor(currentYear))]
      for x in range (qtrs):
          yearColumns.append('Q'+str(((currentQuarter+x) % 4)+1)+'/'+str(math.floor(currentYear+(x+1)*0.25)))
      return yearColumns


  



    



class EnergyCredit(CorporateCredit):
  def __init__(self, name, cusip, ticker, rating, sector, country):      
    super().__init__(name, cusip, ticker, rating, sector, country)

     

 


class EPCredit(EnergyCredit):
  def __init__(self, name, cusip, ticker, rating, sector, country):
    super().__init__(name, cusip, ticker, rating, sector, country)
     # think about what parameters fully define an individual E&P Credit - Cash cost, production for each commodity, realized px, etc 

  def DB_Input(self):
    DebtEBITDA = input("Enter Debt/EBITDA: ")
    CashCost = input("Enter cash cost: ")
    OilProd = input("Enter Daily Oil Prod (bbd/d): ")
    NGProd = input("Enter Daily Natural Gas Prod (mmcf/d):  ")
    NGLProd = input("Enter Daily NGL Prod (bbd/d):  ")

    # conn = sqlite3.connect("test.db")
    # c = conn.cursor()
    # c.execute("INSERT TABLE EandP self.name CashCost OilProd NGLProd NGProd DebtEBITDA)


  def SetHedges(self):
    print(OilPrices[0])
    n = int(input("How many years are hedged: "))
    Hedge = [[0 for i in range(n)] for j in range(7)]
    #[year 0, year 1, ...]
    #[oil volume]
    #[oil realized price]
    #[NGL volume]
    #[NGL realized price]
    #[NG volume]
    #[NG realized price]

    #print(type(n))
    for i in range(0,n):
      print("Year "+str(i+1)+" :")
      TypeHedge = input("Oil hedged through (f)utures or (o)ptions?")

      if TypeHedge=='f':
        Hedge[1][i] = int(input("Oil volume hedged: "))
        Hedge[2][i] = int(input("Hedged Price: "))
      
      elif TypeHedge=='o':
        Hedge[1,i] = int(input("Oil volume hedged: "))
        Floor = int(input("Put Strike: "))
        Ceiling = int(input("Written Call Strike: "))
        SoldPut = int(input("Written Put Strike: "))
        Hedge[2][i] = OilPrices[i] - max(OilPrices[i]-Ceiling,0)+max(Floor-OilPrices[i],0)-max(SoldPut-OilPrices[i],0)

      TypeHedge = input("Gas hedged through (f)utures or (o)ptions?")

      if TypeHedge=='f':
        Hedge[5][i] = int(input("Gas volume hedged: "))
        Hedge[6][i] = int(input("Hedged Price: "))
      
      elif TypeHedge=='o':
        Hedge[5][i] = int(input("Gas volume hedged: "))
        Floor = int(input("Put Strike: "))
        Ceiling = int(input("Written Call Strike: "))
        SoldPut = int(input("Written Put Strike: "))
        Hedge[6][i] = OilPrices[i] - max(OilPrices[i]-Ceiling,0)+max(Floor-OilPrices[i],0)-max(SoldPut-OilPrices[i],0)

      TypeHedge = input("NGL hedged through (f)utures or (o)ptions?")

      if TypeHedge=='f':
        Hedge[3][i] = int(input("NGL volume hedged: "))
        Hedge[4][i] = int(input("Hedged Price: "))
      
      elif TypeHedge=='o':
        Hedge[3][i] = int(input("NGL volume hedged: "))
        Floor = int(input("Put Strike: "))
        Ceiling = int(input("Written Call Strike: "))
        SoldPut = int(input("Written Put Strike: "))
        Hedge[4][i] = OilPrices[n] - max(OilPrices[i]-Ceiling,0)+max(Floor-OilPrices[i],0)-max(SoldPut-OilPrices[i],0)
      
      return(Hedge)
      
      #print(Hedge)









A = EPCredit("Apache","xx","APA",["BBB-","BBB"],"Energy","USA")

A.Print_Description()
#A.DB_Input()
Hedge = A.SetHedges()
print(Hedge)




Name:  Apache
Ticker:  APA
Rating:  ['BBB-', 'BBB']
Sector:  Energy
Country:  USA
40
How many years are hedged: 1
Year 1 :
Oil hedged through (f)utures or (o)ptions?f
Oil volume hedged: 25
Hedged Price: 25
Gas hedged through (f)utures or (o)ptions?f
Gas volume hedged: 25
Hedged Price: 25
NGL hedged through (f)utures or (o)ptions?f
NGL volume hedged: 25
Hedged Price: 25
[[0], [25], [25], [25], [25], [25], [25]]
